If you have images split into folders as used for regression or semantic segmenatation you can create an instance segmentation dataset with the following code

In [ ]:
import os 
import numpy as np
import imageio as io
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt

In [ ]:
def plot(img): 
    plt.imshow(img)
    plt.show()

The following code will copy the image files from the semantic segmentation folder to the instance segmentation folder creating the correct folder sturcture for you

In [ ]:
## Preparing images
path_in = "/docs/examples/SemanticSegmentation/"
path_out = "/docs/examples/InstanceSegmentation/"

image_path_in = path_in +"/train/image/"
image_path_out = path_out +"/train/"

files = os.listdir(image_path_in)
for file in files: 
    data = np.array(imread(image_path_in + file))
    print(np.shape(data))
    filename = file[0:-4]
    filename = filename.replace('.','')
    print(filename)
    outname = (image_path_out + filename + "/image/")
    os.makedirs(outname, exist_ok=True)
    imsave(outname +"/" + filename + ".png", data.astype("uint8"))

The following code will create groundtruth masks for you and save them in the corred folder for instance segmentation.
This code is not useable for any case, it works best for non-overlapping objects. 

In [ ]:
## Preparing Masks for training data
path_in = "/docs/examples/SemanticSegmentation/"
path_out = "/docs/examples/InstanceSegmentation/"

image_path_in = path_in +"/train/groundtruth/"
image_path_out = path_out +"/train/"

files = os.listdir(image_path_in)
for file in files: 
    if os.path.exists(image_path_in + file):
        data = imread(image_path_in + file)
        filename = file[0:-4]
        labels = label(data, background=0) # same image_binary as above
        propsa = regionprops(labels)   
        print("labels", len(regionprops(labels)))
        #plot(data)
        for region_index, region in enumerate(regionprops(labels)):
            print("starting region", region_index, "of", len(regionprops(labels)))
            new_mask = np.zeros(data.shape)
            new_mask[labels == region.label] = 255
            if np.mean(new_mask) > 0.03:
                filename = filename.replace('.','')
                outname = (image_path_out + filename + "/mask/")
                os.makedirs(outname, exist_ok=True)
                imsave(outname + filename + "_ind" + str(region_index) + ".png", new_mask.astype("uint8"))
        '''if no labels could be found, make empty mask'''
        if len(regionprops(labels)) == 0: 
            print("Making empty mask")
            new_mask = np.zeros(data.shape)
            filename = filename.replace('.','')
            outname = (image_path_out + filename + "/mask/")
            os.makedirs(outname, exist_ok=True)
            imsave(outname + filename + "_ind" + str(0) + ".png", new_mask.astype("uint8"))